# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,las tablas,7.7667,-80.2833,86.18,99,93,5.73,PA,1722014514
1,1,doctor arroyo,23.6667,-100.1833,66.56,73,99,1.41,MX,1722014515
2,2,yellowknife,62.4560,-114.3525,74.41,46,40,13.80,CA,1722014483
3,3,coquimbo,-29.9533,-71.3436,58.73,77,0,13.80,CL,1722014515
4,4,puerto ayora,-0.7393,-90.3518,76.96,96,44,8.01,EC,1722014516
...,...,...,...,...,...,...,...,...,...,...
377,377,new martinsville,39.6445,-80.8576,80.92,52,77,1.99,US,1722014632
378,378,yallahs,17.8748,-76.5624,92.93,65,10,14.56,JM,1722014633
379,379,sonoita,31.8500,-112.8333,94.05,33,50,7.65,MX,1722014633
380,380,choix,26.7092,-108.3219,93.16,39,56,2.66,MX,1722014633


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
mapPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = 'OSM',
    frame_width = 896,
    frame_height = 504,
    
)


# Display the map
mapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values #Between 69.8F and 80.6F, No clouds, wind speeds below or equal to 10.0662MPH
# IMPERIAL MEASUREMENTS RAHHH!!!!
FilteredTemps = city_data_df.loc[(city_data_df['Max Temp'] >= 69.8) & (city_data_df['Max Temp'] <= 80.6)]
FilteredWinds = city_data_df.loc[(city_data_df['Wind Speed'] <= 10.0662 )]
FilteredClouds = city_data_df.loc[(city_data_df['Cloudiness'] <=0)]

# Drop any rows with null values
IdealConditions = FilteredClouds.dropna(how='all')

# Display sample data
IdealConditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,coquimbo,-29.9533,-71.3436,58.73,77,0,13.80,CL,1722014515
12,12,north bend,43.4065,-124.2243,67.26,72,0,10.36,US,1722014518
22,22,jamestown,42.0970,-79.2353,75.52,44,0,8.05,US,1722014522
28,28,pimentel,-6.8367,-79.9342,70.09,64,0,13.80,PE,1722014523
30,30,clarinda,40.7419,-95.0383,87.60,62,0,9.22,US,1722014524
...,...,...,...,...,...,...,...,...,...,...
365,365,regina,50.4501,-104.6178,71.71,40,0,17.27,CA,1722014481
367,367,emporia,38.4039,-96.1817,92.52,43,0,8.05,US,1722014629
368,368,silver city,32.7701,-108.2803,84.63,33,0,5.01,US,1722014630
374,374,swift current,50.2834,-107.8014,70.61,35,0,17.27,CA,1722014631


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = IdealConditions.loc[:,['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,coquimbo,CL,-29.9533,-71.3436,77,
12,north bend,US,43.4065,-124.2243,72,
22,jamestown,US,42.0970,-79.2353,44,
28,pimentel,PE,-6.8367,-79.9342,64,
30,clarinda,US,40.7419,-95.0383,62,
...,...,...,...,...,...,...
365,regina,CA,50.4501,-104.6178,40,
367,emporia,US,38.4039,-96.1817,43,
368,silver city,US,32.7701,-108.2803,33,
374,swift current,CA,50.2834,-107.8014,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 8000
category = "accommodation.hotel"
limit = 10

params = {
    "categories": category,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # Not necessary
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    print(row)
# Display sample data
hotel_df

Starting hotel search
coquimbo - nearest hotel: Hotel Iberia
City          coquimbo
Country             CL
Lat           -29.9533
Lng           -71.3436
Humidity            77
Hotel Name            
Name: 3, dtype: object
north bend - nearest hotel: The Mill Casino Hotel
City          north bend
Country               US
Lat              43.4065
Lng            -124.2243
Humidity              72
Hotel Name              
Name: 12, dtype: object
jamestown - nearest hotel: DoubleTree Jamestown
City          jamestown
Country              US
Lat              42.097
Lng            -79.2353
Humidity             44
Hotel Name             
Name: 22, dtype: object
pimentel - nearest hotel: Hospedaje Calle del Sol
City          pimentel
Country             PE
Lat            -6.8367
Lng           -79.9342
Humidity            64
Hotel Name            
Name: 28, dtype: object
clarinda - nearest hotel: Celebrity Inn
City          clarinda
Country             US
Lat            40.7419
Lng           -95

,City,Country,Lat,Lng,Humidity,Hotel Name
3,coquimbo,CL,-29.9533,-71.3436,77,Hotel Iberia
12,north bend,US,43.4065,-124.2243,72,The Mill Casino Hotel
22,jamestown,US,42.0970,-79.2353,44,DoubleTree Jamestown
28,pimentel,PE,-6.8367,-79.9342,64,Hospedaje Calle del Sol
30,clarinda,US,40.7419,-95.0383,62,Celebrity Inn
...,...,...,...,...,...,...
365,regina,CA,50.4501,-104.6178,40,Empire Hotel
367,emporia,US,38.4039,-96.1817,43,Budget Host Inn Emporia
368,silver city,US,32.7701,-108.2803,33,Murray Hotel
374,swift current,CA,50.2834,-107.8014,35,York Hotel & Bar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
mapPlot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = 'OSM',
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["Hotel Name"] 
)
# Display the map
mapPlot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)